In [519]:
import pandas as pd
out=pd.read_table('csdn_blog_100.csv',sep='\t',
                engine='python',index_col=['_id'],header=None,names=['_id','标题','标签','时间','pv','作者','原创','c'])
out['nlp']=''

In [520]:
import jieba
import jieba.analyse
import jieba.posseg as pseg
import codecs
from bs4 import BeautifulSoup
from gensim import corpora, models, similarities

# 加停用词库
stopwords = codecs.open('../words/CNstop_words_zh.txt','r',encoding='utf8').readlines()
stopwordlist = [ w.strip() for w in stopwords ]
# 结巴分词后的停用词性 [标点符号、连词、助词、副词、介词、时语素、‘的’、数词、方位词、代词]
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

In [521]:
def tokenization(doc):
#     结巴分词，去掉停用词
    result = []
    text = doc
    words = pseg.cut(text)
    for word, flag in words: 
        if flag not in stop_flag and word not in stopwords: 
            result.append(word)
    return result

ids=[]
dockeys=[]
# 100篇文档分词，保存
for row in out.itertuples():
    soup = BeautifulSoup(row.c,'html5lib').getText().strip()
    ids.append(row.Index)
    word_list = tokenization(soup)
    dockeys.append(word_list)
    out.loc[out.index==row[0],['nlp']] = ' '.join([w for w in word_list])

out.drop(out.columns[[6]], axis=1,inplace=True)
out.to_csv('csdn_blog_100_nlp.csv',sep='\t')
try:
#     保存文档id列表
    import pickle
    f = open("./csdn_blog_ids_100.dat", "wb")
    pickle.dump(ids, f)
    f.close()
except Exception as e:
    print('csdn_blog_ids_100.dat保存文件出错')
    
try:
#     保存文档dockeys列表
    import pickle
    f = open("./csdn_blog_dockeys_100.dat", "wb")
    pickle.dump(dockeys, f)
    f.close()
except Exception as e:
    print('csdn_blog_dockeys_100.dat保存文件出错')

In [526]:
from gensim import corpora, models, similarities
# 建立词袋模型
dictionary = corpora.Dictionary(dockeys)
dictionary.save('csdn_blog_100.dict')
# print(dictionary)
# print(len(dockeys))
# for doc in dockey:
#     print(len(doc))
# 生成词向量
corpus = [dictionary.doc2bow(text) for text in dockeys]
corpora.MmCorpus.serialize('csdn_blog_100.mm', corpus)
# print('len',len(corpus))
# print(doc_vectors)
# for doc in corpus:
#     print(len(doc))

In [489]:
# 建立TF-IDF模型
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]
# print(len(tfidf_corpus))
# print(len(tfidf_vectors[0]))
# for doc in tfidf_vectors:
#     print(doc)
# 持久化
tfidf.save("./csdn_blog_model_100.tfidf")
tfidf = models.TfidfModel.load("./csdn_blog_model_100.tfidf")

In [492]:
# 生成tf/idf模型,相似度矩阵
index_tfidf = similarities.MatrixSimilarity(tfidf_corpus)
index_tfidf.save('csdn_blog_100.index')

def tfidfsimilarities(f,index,N=20):
    # 求tf/idf模型,相似度
    doc=''
    with open(f, 'r') as f:
        doc = f.read().replace('\n','')
    query=tokenization(doc)
    query_bow=dictionary.doc2bow(query)
    sims = index[query_bow]
    from operator import itemgetter, attrgetter
    return sorted(list(enumerate(sims)), key=itemgetter(1),reverse=True)[0:N]

def toids(sim,ids):
#     语料库ID转换成数据库里的_id
    return [(ids[s[0]],s[1]) for s in sim]

sim1 = tfidfsimilarities('../测试样本/1.txt',index_tfidf)
toids(sim1,ids)

[(4092, 0.17394318),
 (10107, 0.13801867),
 (6112, 0.12311725),
 (4461, 0.11609715),
 (321, 0.10852387),
 (1883, 0.094831996),
 (4926, 0.093249574),
 (4137, 0.079429373),
 (12345, 0.076761037),
 (5670, 0.076752216),
 (10364, 0.07548704),
 (823, 0.070422918),
 (12111, 0.069211349),
 (2606, 0.064810753),
 (12242, 0.063527085),
 (10970, 0.062913775),
 (1823, 0.061469492),
 (4485, 0.060431741),
 (322, 0.057909347),
 (1834, 0.05714741)]

In [493]:
sim1

[(41, 0.17394318),
 (78, 0.13801867),
 (63, 0.12311725),
 (47, 0.11609715),
 (0, 0.10852387),
 (9, 0.094831996),
 (54, 0.093249574),
 (43, 0.079429373),
 (98, 0.076761037),
 (60, 0.076752216),
 (81, 0.07548704),
 (3, 0.070422918),
 (96, 0.069211349),
 (21, 0.064810753),
 (97, 0.063527085),
 (87, 0.062913775),
 (6, 0.061469492),
 (48, 0.060431741),
 (1, 0.057909347),
 (8, 0.05714741)]

In [497]:
sim2 = tfidfsimilarities('../测试样本/2.txt',index_tfidf)
sim3 = tfidfsimilarities('../测试样本/3.txt',index_tfidf)
sim4 = tfidfsimilarities('../测试样本/4.txt',index_tfidf)
sim5 = tfidfsimilarities('../测试样本/5.txt',index_tfidf)

In [498]:
# 构建LSI模型，设置主题数为4
# lsi = models.LsiModel(doc_vectors, id2word=dictionary, num_topics=4)
# lsi.print_topics(4)
# lsi_vector = lsi[doc_vectors]
# for vec in lsi_vector:
#     print(vec)

In [499]:
# 求LSI模型相似度
# def tfidfsimilarities(f,lsivectors):
#     doc=''
#     with open(f, 'r') as f:
#         doc = f.read().replace('\n','')
#     query=tokenization(doc)
#     query_bow = dictionary.doc2bow(query)
#     lsi_bow=lsi[query_bow]
#     # lsi模型,相似度
#     index = similarities.MatrixSimilarity(lsivectors)
#     sims = index[lsi_bow]
#     from operator import itemgetter, attrgetter
#     return sorted(list(enumerate(sims)), key=itemgetter(1),reverse=True)[0:20]
# sim1 = tfidfsimilarities('../测试样本/1.txt',tfidf_vectors)
# sim2 = tfidfsimilarities('../测试样本/2.txt',tfidf_vectors)
# sim3 = tfidfsimilarities('../测试样本/3.txt',tfidf_vectors)
# sim4 = tfidfsimilarities('../测试样本/4.txt',tfidf_vectors)
# sim5 = tfidfsimilarities('../测试样本/5.txt',tfidf_vectors)